# Adaptive PDE discretizations on cartesian grids 
## Volume : Divergence form PDEs
## Part : Linear elasticity
## Chapter : Dirichlet energy

We present a new discretization of the Dirichlet energy arising in linear elasticity, associated with a positive definite Hooke tensor, otherwise arbitrary. The approach is based on Selling's decomposition of the Hooke tensor, and for this reason is only applies in dimension two at the moment.

The Dirichlet energy of linear elasitcity, defined for (small) displacement maps $v : \Omega \to R^d$, reads
$$
    E(v) := \int_\Omega \sum_{ijkl} c_{ijkl}(x) \epsilon_{ij}(x) \epsilon_{kl}(x) \ dx,
$$
where the indices $i,j,k,l$ range from $0$ to $d-1$. We denoted by $c_{ijkl}(x)$ the Hooke tensor at a point $x \in \Omega$, and introduced the symmetrized gradient of the displacement field, also known as the strain tensor $\epsilon$
$$
    \epsilon_{ij}(x) = \frac 1 2 \Big (\frac{\partial v_i}{\partial x_j} + \frac{\partial v_j}{\partial x_i} \Big).
$$

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"../..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; print(TocTools.displayTOC('ElasticityDirichlet','FMM'))

In [2]:
from agd import LinearParallel as lp
from agd import FiniteDifferences as fd
from agd.Metrics.Seismic import Hooke
from agd import AutomaticDifferentiation as ad
from agd import Domain
from agd.Plotting import savefig; #savefig.dirName = 'Images/ElasticityDirichlet'
norm_infinity = ad.Optimization.norm_infinity

In [3]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from copy import copy

In [4]:
arr0=ad.Dense.identity((2,2))
arr0.reshape((4,))

denseAD(array([0., 0., 0., 0.]),
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]))

In [5]:
a=[1,2]
tuple(a)

(1, 2)

In [6]:
arr0=ad.Dense.identity((2,))
arr1=np.zeros((2,))
ad.stack((arr0,arr1),axis=0)

denseAD(array([[0., 0.],
       [0., 0.]]),
array([[[1., 0.],
        [0., 1.]],

       [[0., 0.],
        [0., 0.]]]))

In [10]:
a=[1,2]
a.append(3)
a

[1, 2, 3]

In [13]:
tuple((2,))

(2,)

In [14]:
tuple(*[1,2],*[3,4])

TypeError: tuple expected at most 1 arguments, got 4

In [18]:
hasattr(1,"__iter__")

False

## 1. Decomposition of a hooke tensor

A Hooke tensor defines a quadratic form on the set of symmetric matrices $\epsilon \in S_d$
$$
    c(\epsilon,\epsilon) = \sum_{ijkl} c_{ijkl} \epsilon_{ij} \epsilon_{kl}.
$$
Note that $S_d$ has dimension $D=d (d+1)/2$. We limit our attention to the case $d=2$, since the case $d=1$ is excessively trivial, and the case $d=3$ would require an implementation of the $6$-dimensional Voronoi reduction, which we do not have at this stage.

We use Selling's decomposition to rewrite this quadratic form as
$$
    c(\epsilon,\epsilon) = \sum_r \rho_r {\rm Tr}(\epsilon m_r)^2
$$
where $\rho_r \geq 0$, $m_r \in S_2(Z)$ is a symmetric matrix with integer coordinates, and $0 \leq r < D (D+1)/2=6$. For that purpose, we rely on Selling's decomposition of the Hooke tensor, which applies in dimension since the linear space of $2\times 2$ symmetric matrices has dimension $3$.

<!--- In the following, we denote as usual $c(\epsilon) := c(\epsilon,\epsilon)$. --->

The stress tensor $\sigma$ depends linearly on the strain tensor $\epsilon$, for a given hooke tensor $c$, and is characterized by the identity
$$
    {\rm Tr}(\sigma \epsilon) = c(\epsilon,\epsilon).
$$
With the correct index conventions, one has $\sigma_{ij} = \sum_{kl} c_{ijkl} \epsilon_{kl}$, or simply $\sigma = c \epsilon$.

### 1.1 Generic tensor

We illustrate the decomposition on a generic tensor, describing the anisotropic elasticity of a mica rock medium, whose layers are rotated.

In [4]:
metric = Hooke.mica().extract_xz() 
metric.rotate_by(0.5)

Selling's decomposition involves $D=6$ weights and offsets, in dimension $d=2$. 

In [5]:
coefs,moffsets = metric.Selling()

In [6]:
coefs

array([ 4.39753907, 31.3882811 ,  3.86972663, 17.65302611, 41.16677899,
       29.81855447])

The offsets are presented as symmetric matrices, with integer entries.

In [7]:
moffsets.shape

(2, 2, 6)

In [8]:
for i in range(6): print(moffsets[...,i],"\n")

[[1 1]
 [1 1]] 

[[-1 -1]
 [-1  0]] 

[[2 1]
 [1 1]] 

[[ 0  0]
 [ 0 -1]] 

[[-1  0]
 [ 0  0]] 

[[1 0]
 [0 1]] 



In order to check the correctness of the decomposition, let us introduce an arbitrary $2\times 2$ symmetric matrix, and evaluate $c(m)$ using the original expression and the decomposition.

In [9]:
m = np.array(((1.5,2.3),(2.3,3.9)))

In [10]:
sum_hooke = metric.dot_AA(m) 

In [11]:
mm = fd.as_field(m,coefs.shape)
sum_inner = (coefs*lp.trace(lp.dot_AA(mm,moffsets))**2).sum(axis=0)

In [12]:
assert(np.abs(sum_hooke-sum_inner) < 1e-12)

### 1.2 Isotropic tensor

Isotropic elasticity tensors only have two degrees of freedom. Without loss of generality, we consider the Lamé parameters. These parameters relate the strain tensor $\epsilon$ with the stress tensor $\sigma$
$$
    \sigma = 2 \mu \epsilon + \lambda {\rm Tr}(\epsilon) {\rm Id},
$$
and the quadratic form reads 
$$
    c(\epsilon,\epsilon) = 2 \mu {\rm Tr}(\epsilon^2) + \lambda {\rm Tr}(\epsilon)^2.
$$

In [13]:
metric = Hooke.from_Lame(1,2.)
print(f"""An isotropic hooke tensor :\n{metric.hooke}\n""")

An isotropic hooke tensor :
[[5. 1. 0.]
 [1. 5. 0.]
 [0. 0. 2.]]



In [14]:
print(f"""Isotropic Hooke tensors are linear combinations of: \n{Hooke.from_Lame(1.,0.).hooke}\n"""
      f"""and \n{Hooke.from_Lame(0.,1.).hooke}\n""")

Isotropic Hooke tensors are linear combinations of: 
[[1. 1. 0.]
 [1. 1. 0.]
 [0. 0. 0.]]
and 
[[2. 0. 0.]
 [0. 2. 0.]
 [0. 0. 1.]]



As their name suggests, isotropic hooke tensors are invariant under rotations.

In [15]:
metric_rot = copy(metric)
metric_rot.rotate_by(0.5)
assert(norm_infinity(metric.hooke-metric_rot.hooke)<1e-15)

Selling's decomposition of an isotropic Hooke tensor is very structured and predictable. It involves offsets $m_r$ which are independent of the parameters $(\lambda,\mu)$, and weights $\rho_r(\lambda,\mu)$ depending linearly on the Lame parameters. In addition, several of these coefficients vanish.

In [16]:
coefs,moffsets = metric.Selling()

In [17]:
coefs

array([-0., -0.,  2.,  4.,  4.,  1.])

In [18]:
for c,o in zip(coefs,np.moveaxis(moffsets,-1,0)): 
    if c!=0.:
        print(f"coef:{c}, moffset:\n{o}\n")

coef:2.0, moffset:
[[0 1]
 [1 0]]

coef:4.0, moffset:
[[ 0  0]
 [ 0 -1]]

coef:4.0, moffset:
[[-1  0]
 [ 0  0]]

coef:1.0, moffset:
[[1 0]
 [0 1]]



In this special isotropic case, we expect to recover known finite difference schemes.

## 2. Finite difference energy

We approximate the linear elastic energy using a second order accurate finite differences scheme, which exploits our tensor decomposition. The scheme is based on the identity
$$
    c(\sigma,\sigma) = \sum_r \rho_r {\rm Tr}(m_r \nabla v)^2
$$
where $(\rho_r,m_r)$ is Selling's decomposition of $c$. We could replace $\sigma$ with $\nabla v$ thanks to the symmetry of $m_r$.

Then we use the finite difference approximations
$$
    {\rm Tr}(m \nabla v) = \sum_{0 \leq i < d} \frac{v_i(x+h \epsilon_i m[i] )-v_i(x)}{h \epsilon_i} + O(h),
$$
where $\epsilon_i$, $0\leq i < d$ are arbitrary signs, and $m[i]$ denotes the $i$-th column of $m$ (which is a symmetric matrix). Squaring this expression, averaging over all possible sign choices, and summing with weights $\rho_r$, we obtain a second order consistent approximation of the local linear elastic energy.
$$
    c(\sigma,\sigma) = \sum_{0 \leq r \leq D (D+1)/2} \frac{\rho_r}{2^d} \sum_{\epsilon \in \{-1,1\}^d}  
    \Big(\sum_{0 \leq i < d} \frac{v_i(x+h \epsilon_i m_r[i] )-v_i(x)}{h \epsilon_i}\Big)^2.
$$

**Remark**
If the coordinates of $m_r[i]$ are not co-prime, for some $0 \leq r < D (D+1)/2$ and $0 \leq i < d$, then one can improve the scheme taking advantage of this fact in the finite differences.

In [19]:
def Energy(hooke,v,dom):
    """
    Linear elastic energy density, associated with a Hooke tensor, and a displacement field.
    Discretization using finite differences, and Selling's decomposition of the Hooke tensor.
    """
    assert(len(v)==2)
    coefs,moffsets = hooke.Selling()
    dvp = tuple( dom.DiffUpwind(v[i], moffsets[i]) for i in range(2))
    dvm = tuple(-dom.DiffUpwind(v[i],-moffsets[i]) for i in range(2))
    
    # Consistent approximation of Tr(moffset*grad(v))
    dv  = ad.array((dvp[0]+dvp[1],dvp[0]+dvm[1],dvm[0]+dvp[1]+dvm[0]+dvm[1]))
    dv2 = np.sum(dv**2,axis=0) / 4.
    return (coefs*dv2).sum(axis=0) 

For comparison, let us also evaluate the elastic energy using automatic differentiation.

In [20]:
def Energy_ad(hooke,v,X,h):
    """
    Linear elastic energy density, associated with a hooke tensor and a displacement field.
    The latter must be given as a function, compatible with automatic differentiation.
    """
    # Differentiate the displacement field
    X_ad = ad.Dense.identity(constant=X,shape_free=(2,))
    grad = v(X_ad).gradient()
    eps = 0.5*(grad+lp.transpose(grad))
    return hooke.dot_AA(eps) 

Let us observe the convergence of the finite element energy toward the ad energy in a smooth periodic setting.

In [21]:
def v(X):
    x0,x1 = X*(2.*np.pi)
    return ad.array((np.cos(x0) - 2.*np.sin(x1),np.cos(x0+2*x1)))

def hooke(X):
    x0,x1 = X*(2.*np.pi)
    metric = Hooke.mica().extract_xz()
    metric.rotate_by(0.3*np.sin(x0)+0.5*np.cos(x1))
    return metric

In [22]:
aX,h = np.linspace(0,1,200,retstep=True)
X=np.array(np.meshgrid(aX,aX,indexing='ij'))
dom = Domain.MockDirichlet(X.shape,h,padding=None) #Periodic domain (wrap instead of pad)

In [23]:
energy_density_ad = Energy_ad(hooke(X),v,X,h)
energy_density_fd = Energy(hooke(X),v(X),dom)

TypeError: 'list' object cannot be interpreted as an integer

In [24]:
energy_ad = energy_density_ad.sum()
energy_fd = energy_density_fd.sum()
print(f"Relative error: {(energy_ad-energy_fd)/energy_fd}")

NameError: name 'energy_density_ad' is not defined

In [ ]:
help(Domain.MockDirichlet)

In [ ]:
np.__version__